In [ ]:
import os
import torch

from utils.callbacks import EarlyStopping, History


from utils.utils_data import get_train_dataloaders, get_test_dataloader
from models.DVBCDModel import DVBCDModel

print(torch.__version__)
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
torch.autograd.set_detect_anomaly(True)

USE_GPU = True
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
if USE_GPU and torch.cuda.is_available():
    DEVICE = torch.device('cuda:3')
else:
    DEVICE = torch.device('cpu')
print('using device:', DEVICE)

In [ ]:
#args.train_data_path = '/data/BasesDeDatos/Camelyon/Camelyon17/training/Toy/'
data_path = '/data/BasesDeDatos/Camelyon/Camelyon17/training/patches_224/'

#dataloaders = get_dataloaders(args.data_path, args.patch_size, args.batch_size, args.num_workers, val_prop=0.3)
train_dataloader, val_dataloader = get_train_dataloaders(data_path, 224, 16, 64, val_prop=0.2, n_samples=2000)

print(len(train_dataloader), len(val_dataloader))

In [ ]:
model = DVBCDModel(
                cnet_name="unet_6", mnet_name="resnet_18_in", 
                sigmaRui_sq=torch.tensor([0.05, 0.05]), theta=0.5, lr_cnet=1e-4, 
                lr_mnet=1e-4, lr_decay=0.1, clip_grad_cnet=1e5, clip_grad_mnet=1e5,
                device=DEVICE
                )
model.set_callbacks([EarlyStopping(model, "val_loss_mse", patience=2), History()])
model.fit(5, train_dataloader, val_dataloader, pretraining=False)